In [1]:
import numpy as np

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import row, column
from bokeh.models import Legend

from scipy.optimize import root

output_notebook(hide_banner=True)

# Context

Here we are concerned with solving the following Initial Value Problem (IVP):
\begin{equation}
\left\{
\begin{aligned}
x^\prime(t)&=f\left(t,x(t)\right), \quad t\geq t_0\\
x(t_0)&= x_0
\end{aligned}
\right.
\end{equation}
with $x : t \mapsto x(t) \in \mathbb{R}^m$, $m \in \mathbb{N}^*$.

It is assumed that the considered IVP possesses a unique solution on some interval $[t_0,t_f]$ with $t_f>t_0$

To approximate the solution of this IVP we consider Linear Multistep Methods (LMM). In the first part (I) of this notebook, we will compare some 2-step LMM with several methods decribed in TD2 (TS2, TR). Part (II) is concerned with a more general implementation of 2-step explicit LMM and part (III) is the equivalent of part (II) for implicit methods. 

All examples and methods described here are adapted from: Giffiths and Higham, *Numerical Methods for Ordinary Differential Equations*, 2010, Springer.

**Simplifiying assumption**: In the present notebook only scalar problems are considered to keep relatively simple implementations. We also assume that step size $h$ is always such that $t_f-t_0 = Nh$ with $N \in \mathbb{N}$. One should always remember that if this condition does not hold then the methods presented here will not reach the correct final time $t_f$.

# Part I: Introduction to LMM, Adam-Bashforth method

In this part we focus on a specific IVP, denoted IVP (1), given by :
$$
f_1(t,x) = (1-2t)x,\quad x(0) = 1.
$$
The solution to this IVP is
$$
x_{sol}^1 : t \mapsto \exp \left(\frac{1}{4} - \left(t-\frac{1}{2}\right)^2\right)
$$

Our goal here is to compare, on this problem, a first LMM introduced in the last lecture with the Euler explicit, the Trapezoidal rule and TS2 methods.

For the purpose of using the Taylor series method, the right hand side of the IVP must be differentiated with respect to $t$ so we introduce
$$
g_1(t,x) = \partial_t f_1(t,x) + \partial_x f_1(t,x)f_1(t,x) = -2 x + (1-2t)(1-2t)x = \left((1-2t)^2-2\right) x
$$
with $(t,x)\in \mathbb{R}_+\times\mathbb{R}$.

We also recall that, in the specific case of IVP 1, the Trapezoidal rule reads, for $n \in \mathbb{N}$, $t_n = t_0 + nh$:
$$
x_{n+1}-x_n =  \frac{h}{2}\left(f_1(t_{n+1},x_{n+1})+f_1(t_n,x_n)\right) = x_n+\frac{h}{2}\left[(1-2t_{n+1})x_{n+1}+(1-2t_n)x_n\right]
$$
so that we have
$$
x_{n+1} = \frac{1+\frac{h}{2}(1-2t_n)}{1-\frac{h}{2}(1-2t_{n+1})}x_n
$$
*We emphasize that the trapezoidal rule is implicit in general, here one is able to find an analytical explicit formula only because of the specific structure of IVP (1).*

In [2]:
def f1(t,x):
    return (1-2*t)*x

def x1_exa(t):
    return np.exp(1/4-(1/2-t)**2)

def g1(t,x):
    return ((1-2*t)**2-2)*x

**We recall here the Euler, Trapezoidal and TS2 methods. The methods return an array of size $nt$ where $nt = \left\lfloor\frac{t_f-t_0}{h}\right\rfloor+1$.**

In [3]:
def Euler(f, t0, tf, h, x0):
    
    nt   = int(round((tf-t0)/h))+1 #number of columns of the output array
    t    = t0                      #initial time
    x    = np.zeros((nt,))         
    x[0] = x0                      #initial value
    it   = 0

    for it in range(nt-1):
        x[it+1] = x[it] + h*f(t0+it*h,x[it]) 

    return x

In [4]:
def TS2(f, g, t0, tf, h, x0):
    
    nt   = int(round((tf-t0)/h))+1 #number of columns of the output array
    t    = t0                      #initial time
    x    = np.zeros((nt,))
    x[0] = x0                      #initial value
    it   = 0
    
    for it in range(nt-1):
        x[it+1] = x[it] + h*f(t0+it*h,x[it]) + h**2/2*g(t0+it*h, x[it])

    return x

In [5]:
def TrapRule1(t0, tf, h, x0): #Method specific to the IVP (1)
    
    nt   = int(round((tf-t0)/h))+1 #number of column of the output array
    t    = t0                      #intial time
    x    = np.zeros((nt,))
    x[0] = x0                      #intial value
    it   = 0
    
    for it in range(nt-1):
        tn = t0+it*h
        tnp1 = t0+(it+1)*h
        x[it+1] = (1+h/2*(1-2*tn))/(1-h/2*(1-2*tnp1))*x[it]
        
    return x

We recall that for IVP (1), the Adams-Bashforth method of order two is defined by the iterative formula:
$$
\begin{aligned}
x_{n+2}&= x_{n+1} + \frac{h}{2}\left(3f_1(t_{n+1},x_{n+1})-f_1(t_n,x_n)\right)\\
&=x_{n+1}+\frac{h}{2}\left[3(1-2t_{n+1})x_{n+1}-(1-2t_n)x_n\right]\\
&=\left[1+\frac{3}{2}h(1-2t_{n+1})\right]x_{n+1}-\frac{h}{2}\left[1-2t_n\right]x_n
\end{aligned}
$$
The two step structure of the scheme naturally brings the question of how to compute the second initial value $x_1$ that is required to properly initialized the scheme. In general one do not have access to the exact value $x(t_1)$, and therefore an approximate value must be computed. In what follows we consider several ways to make the first-step approximation and study its impact on the overall convergence of the method.

**Q1. Implement the second-order version of the Adams-Bashforth (AB) method for the IVP (1). Three versions should be implemented:**
- **one version using a constant approximation to obtain the second initial value $x_1$ (i.e. $x_1=x_0$)**
- **one version using the Euler method to obtain the second initial value $x_1$**
- **one version using the Trapezoidal rule to obtain the second initial value $x_1$.**

**The methods should return an array of size $nt$ where $nt = \left\lfloor\frac{t_f-t_0}{h}\right\rfloor+1$.**

In [20]:
#AB method using a constant approximation x_1=x_0 for the first step
def AB_C1(t0, tf, h, x0): #Method specific to IVP (1)
    
    nt = int(round((tf-t0)/h))+1 #number of columns of the output array
    x = np.zeros((nt,)) 
    t = [t0 + i*h for i in range(nt)] 
    
    for i in range(nt): 
        if (i == 0): 
            x[i] = x0 
        elif (i == 1): 
            x[i] = x0 
        else: 
            x[i] = (1 + (3/2)*h*(1-2*t[i-1]))*x[i-1] - (h/2)*(1-2*t[i-2])*x[i-2] 

    return x

In [21]:
#AB method using the Euler method for the first step
def AB_E1(f, t0, tf, h, x0): #Method specific to IVP (1)
    
    nt = int(round((tf-t0)/h))+1 #number of columns of the output array
    x = np.zeros((nt,))
    t = [t0 + i*h for i in range(nt)] 
    
    for i in range(nt): 
        if (i == 0): 
            x[i] = x0 
        elif (i == 1): 
            res = Euler(f, t0, tf, h, x0) 
            x[i] = res[i] 
        else: 
            x[i] = (1 + (3/2)*h*(1-2*t[i-1])) * x[i-1] - (h/2)*(1-2*t[i-2])*x[i-2] 

    return x

In [22]:
#AB method using the Trapezoidal rule method for the first step
def AB_T1(t0, tf, h, x0): #Method specific to IVP (1)
    
    nt = int(round((tf-t0)/h))+1 #number of columns of the output array
    x = np.zeros((nt,))
    t = [t0 + i*h for i in range(nt)] 
    
    for i in range(nt): 
        if (i == 0): 
            x[i] = x0 
        elif (i == 1): 
            res = TrapRule1(t0, tf, h, x0) 
            x[i] = res[i] 
        else: 
            x[i] = (1 + (3/2)*h*(1-2*t[i-1])) * x[i-1] - (h/2)*(1-2*t[i-2])*x[i-2] 
    
    return x

**Q2. As a first step to verify that your implementation is correct, compare these methods on a graph. Also check the global error term on the second graph. You do not need to write more code. If the code you wrote for Q1 is correct, the code below should run and produce what is expected. Please write a small comment telling us whether you are satisfied with what you observe or not.**

In [23]:
t0 = 0.
tf = 4.0
h = 0.2
x0 = 1.

x_Elr    = Euler(f1, t0, tf, h, x0)
x_TS2    = TS2(f1, g1, t0, tf, h, x0)
x_TR1    = TrapRule1(t0, tf, h, x0)
x_ABC1   = AB_C1(t0, tf, h, x0)
x_ABE1   = AB_E1(f1, t0, tf, h, x0)
x_ABT1   = AB_T1(t0, tf, h, x0)

nt = np.size(x_Elr)
t = t0+1.*h*np.arange(nt)

N = 100 # number of points for the exact solution
texa = np.linspace(t0,tf,N)
xexa1 = x1_exa(texa)

fig1 = figure(x_range=(t0, tf), width=980, height=400, x_axis_label='t', y_axis_label='x(t)')
fig1.line(texa,xexa1,legend_label = 'Exact solution')
fig1.circle(t,x_Elr, color = 'navy', legend_label = 'Euler method approximation', size = 8)
fig1.circle(t,x_TS2,fill_color = 'white', legend_label = 'TS method approximation', size = 8)
fig1.x(t,x_TR1,color = 'red', legend_label = 'Trapezoidal rule method approximation', size = 8)
fig1.x(t,x_ABC1,color = 'fuchsia', legend_label = 'AB (C) method approximation', size = 8)
fig1.x(t,x_ABE1,color = 'orange', legend_label = 'AB (E) method approximation', size = 8)
fig1.circle(t,x_ABT1,color = 'red',fill_color='white', legend_label = 'AB (T) method approximation', size = 8)

xexa2 = x1_exa(t)
fig2 = figure(x_range=(t0, tf), width=980, height=400, x_axis_label='t', y_axis_label='GE')
fig2.circle(t,abs(xexa2-x_Elr), color = 'navy', legend_label = 'Euler method approximation', size = 8)
fig2.circle(t,abs(xexa2-x_TS2),fill_color = 'white', legend_label = 'TS method approximation', size = 8)
fig2.x(t,abs(xexa2-x_TR1),color = 'red', legend_label = 'Trapezoidal rule method approximation', size = 8)
fig2.x(t,abs(xexa2-x_ABC1),color = 'fuchsia', legend_label = 'AB (C) method approximation', size = 8)
fig2.x(t,abs(xexa2-x_ABE1),color = 'orange', legend_label = 'AB (E) method approximation', size = 8)
fig2.circle(t,abs(xexa2-x_ABT1),color = 'red',fill_color='white', legend_label = 'AB (T) method approximation', size = 8)

show(column(fig1,fig2))

### Convergence study

We recall that a discrete equivalent of the $L^\infty$-norm $||u||_{L^\infty\left([t_0,t_f]\right)} = \max_{t\in [t_0,t_f]} \left|u(t)\right|$ of a continuous function $u\in \mathcal{C}^0\left([t_0,t_f]\right)$ can be defined for a sequence $(x_j)_{j\in\{0,...,N\}}$ by:
$$
\max_{j\in\{0,...,N\}} \left|x_j\right|.
$$
It is called the $l^\infty$-norm.
To assess the accuracy of a numerical approximation $\left(u_n\right)_{n\in\{0,...,N\}}$, it can be useful to evaluate the following quantity: (error in discrete $l^\infty$-norm)
$$
e_{l^\infty\text{-}norm} = \max_{n\in\{0,...,N\}}\left|u(t_n)-u_n\right|^2
$$
with $t_n = t_0 + nh$ where $h$ is the step size, $n\in\{0,...,N\}$ and $N = \left\lfloor\frac{t_f-t_0}{h}\right\rfloor$.
We shall want to show that $e_{l^\infty-norm} = \mathcal{O}\left(h^p\right)$ for an approximation of order $p$. This is equivalent to $|e_{l^\infty\text{-}norm}|\leq C h^p$ with $C>0$.
For convergence studies, we usually use log scales to show that this upper bound holds:
$$
\ln |e_{l^\infty\text{-}norm}|\leq p \ln (h) + \ln (C)
$$
Indeed, under this formalism, satisfying this inequality is graphically equivalent to remaining below a straight line.

**Q3. To complete the process of checking the validity of your methods, study the order of convergence in terms of $l^\infty$-norm for the implemented methods using log-scale graphs.
You do not need to write more code. If the code you wrote for Q1 is correct, the code below should run and produce what is expected.**

**Do all the choices for the approximation of the first step $x_1$ yield the expected order? Why?**

Three methods : 1) x1 = x0. 2) x1 = x0 + h f(t0, x0). 3) x1 = x0 + (h/2) (f(t0, x0) + f(t1, x1)). 

The error term : 1) x(t1) - x0 = x(t0) + h x'(t0) + O(h^2) - x(t0) = O(h) which is order 1 for the first step. 2) x(t1) - x0 - h f(t0, x0) = O(h^2) which is order 2 for the first step. 

The order of error in the first step keeps in the algorithm. 

However, for Euler method, its global error is e_n = u(tn) - un = O(N h^2) = O(h) with N = T/h. Hence, it is order 1. 

In [24]:
t0 = 0.
tf = 4.0
x0 = 1.

nh = 10
h_values = 10**np.linspace(-4,-1,nh)
err_values = np.zeros((6,nh))
ith=0

for h in h_values:
    x_Elr    = Euler(f1, t0, tf, h, x0)
    x_TS2    = TS2(f1, g1, t0, tf, h, x0)
    x_TR1    = TrapRule1(t0, tf, h, x0)
    x_ABC1   = AB_C1(t0, tf, h, x0)
    x_ABE1   = AB_E1(f1, t0, tf, h, x0)
    x_ABT1   = AB_T1(t0, tf, h, x0)
    
    nt = np.size(x_Elr)
    t = t0+1.*h*np.arange(nt)
    xexa = x1_exa(t)
    err_values[0,ith] = np.amax(np.abs(xexa-x_Elr))
    err_values[1,ith] = np.amax(np.abs(xexa-x_TS2))
    err_values[2,ith] = np.amax(np.abs(xexa-x_TR1))
    err_values[3,ith] = np.amax(np.abs(xexa-x_ABC1))
    err_values[4,ith] = np.amax(np.abs(xexa-x_ABE1))
    err_values[5,ith] = np.amax(np.abs(xexa-x_ABT1))
    ith+=1

fig3 = figure( x_axis_type="log", y_axis_type="log", width=980, height=400, x_axis_label='h', y_axis_label='GE')
fig3.circle(h_values,err_values[0,:], color = 'navy', legend_label = 'Euler method approximation', size = 8)
fig3.circle(h_values,err_values[1,:],fill_color = 'white', legend_label = 'TS method approximation', size = 8)
fig3.x(h_values,err_values[2,:],color = 'red', legend_label = 'Trapezoidal rule method approximation', size = 8)
fig3.x(h_values,err_values[3,:],color = 'fuchsia', legend_label = 'AB (C) method approximation', size = 8)
fig3.x(h_values,err_values[4,:],color = 'orange', legend_label = 'AB (E) method approximation', size = 8)
fig3.circle(h_values,err_values[5,:],color = 'red',fill_color='white', legend_label = 'AB (T) method approximation', size = 8)

fig3.line(h_values,h_values,legend_label = 'order 1 slope')
fig3.line(h_values,h_values**2,color ='red' ,legend_label = 'order 2 slope')
fig3.legend.location = "bottom_right"
show(fig3)

# Part II: Explicit 2-step methods

We recall that a general explicit 2-step method is of the form, for $n\in\mathbb{N}$:
$$
x_{n+2} + \alpha_1 x_{n+1} + \alpha_0 x_n = h\left(\beta_1 f(t_{n+1},x_{n+1})+\beta_0 f(t_n,x_n)\right)
$$
For this method to be consistent, it is required that the following equalities hold:
$$
1+\alpha_1+\alpha_0 = 0, \quad 2 +\alpha_1 = \beta_1+\beta_0
$$

**Bonus: Show that these two conditions are indeed required for the method to be consistent.**

**Q4. Implement a general method that takes as argument the IVP function $f$, the initial and final times $t_0$ and $t_f$, the step size $h$, the starting point $x_0$ and method coefficients $a_k = [\alpha_0,\alpha_1]$ and $b_k = [\beta_0, \beta_1]$. The first step $x_1$ of the method should be obtained thanks to Euler's explicit method.**

In [25]:
def LMMexp(f, t0, tf, h, x0, ak, bk): #The method is no longer specific to IVP (1)
    
    nt = int(round((tf-t0)/h))+1 #number of column of the output array
    x = np.zeros((nt,))
    t = [t0 + i*h for i in range(nt)] 
    
    for i in range(nt): 
        if (i == 0): 
            x[i] = x0 
        elif (i == 1): 
            res = Euler(f, t0, tf, h, x0) 
            x[i] = res[i] 
        else: 
            a0, a1 = ak 
            b0, b1 = bk 
            x[i] = - a1*x[i-1] - a0*x[i-2] + h*(b1*f(t[i-1], x[i-1]) + b0*f(t[i-2], x[i-2])) 
    
    return x

Now we also consider the IVP (2) defined by $f_2(t,x) = x$ and $x(0) = 1$. The solution to this very simple IVP is $x(t) = \exp(t)$, for all $t\in \mathbb{R}_+$

In [26]:
def f2(t,x):
    return x
def x2_exa(t):
    return np.exp(t)
def g2(t,x):
    return x

**Q5. Determine the coefficients $a_k$ and $b_k$ for the Euler method and the AB method. Using a code similar to that of Q2, display the result for IVP (1) and (2).**

For the Euler method x[i] = x[i-1] + hf(t[i-1], x[i-1]), we have that ak = (a0, a1) = (0, -1) and bk = (b0, b1) = (0, 1). 
For the AB method, we have that ak = (a0, a1) = (0, -1) and bk = (b0, b1) = (-1/2, 3/2). 

In [31]:
t0 = 0.
tf = 4.0
h = 0.2
x0 = 1.

##### f2 ##### 
ak2 = (0, -1) 
bk2 = (-1/2, 3/2) 

x_Elr  = Euler(f2, t0, tf, h, x0) 
x_LMM  = LMMexp(f2, t0, tf, h, x0, ak2, bk2) 

nt = np.size(x_Elr)
t = t0+1.*h*np.arange(nt)

N = 100 # number of points for the exact solution
texa = np.linspace(t0,tf,N)
xexa1 = x2_exa(texa)

fig1 = figure(x_range=(t0, tf), width=980, height=400, x_axis_label='t', y_axis_label='x(t)')
fig1.line(texa,xexa1,legend_label = 'Exact solution')
fig1.circle(t,x_Elr, color = 'navy', legend_label = 'Euler method approximation', size = 8)
fig1.circle(t,x_LMM, fill_color = 'white', legend_label = 'LMMexp method approximation', size = 8)

xexa2 = x2_exa(t)
fig2 = figure(x_range=(t0, tf), width=980, height=400, x_axis_label='t', y_axis_label='GE')
fig2.circle(t,abs(xexa2-x_Elr), color = 'navy', legend_label = 'Euler method approximation', size = 8)
fig2.circle(t,abs(xexa2-x_LMM), fill_color = 'white', legend_label = 'LMMexp method approximation', size = 8)

show(column(fig1,fig2)) 

**Q6. Using a code similar to that of Q3, study the order of convergence of these two methods for IVP (1) and (2).**

In [32]:
t0 = 0.
tf = 4.0
x0 = 1.

nh = 10
h_values = 10**np.linspace(-4,-1,nh)
err_values = np.zeros((6,nh))
ith=0

##### f2 ##### 
ak2 = (0, -1) 
bk2 = (-1/2, 3/2) 

for h in h_values:
    x_Elr  = Euler(f2, t0, tf, h, x0)
    x_LMM  = LMMexp(f2, t0, tf, h, x0, ak2, bk2) 

    nt = np.size(x_Elr)
    t = t0+1.*h*np.arange(nt)
    xexa = x2_exa(t)
    err_values[0,ith] = np.amax(np.abs(xexa-x_Elr))
    err_values[1,ith] = np.amax(np.abs(xexa-x_LMM))
    ith+=1

fig3 = figure( x_axis_type="log", y_axis_type="log", width=980, height=400, x_axis_label='h', y_axis_label='GE')
fig3.circle(h_values,err_values[0,:], color = 'navy', legend_label = 'Euler method approximation', size = 8)
fig3.circle(h_values,err_values[1,:],fill_color = 'white', legend_label = 'LMMexp method approximation', size = 8)

fig3.line(h_values,h_values,legend_label = 'order 1 slope')
fig3.line(h_values,h_values**2,color ='red' ,legend_label = 'order 2 slope')
fig3.legend.location = "bottom_right"
show(fig3)

# Part III: Implicit 2-step methods

## Backward Euler method and Trapezoidal rule

The backward Euler method is defined by the following:
$$
x_{n+1} = x_n + h  f(t_{n+1},x_{n+1})
$$
where $t_n = t_0+hn$, $n \in \mathbb{N}$.
In general there is no analytical way to find $x_{n+1}$ and one must approximate the root of the function:
$$
g_n^{Eul} : x \mapsto x-x_n-hf(t_{n+1},x)
$$
Similarly, for the Trapezoidal rule we have:
$$
x_{n+1} = x_n + \frac{h}{2} \left(f(t_{n+1},x_{n+1})+f(t_n,x_n)\right)
$$
and we need to find the root of the function:
$$
g_n^{TR} : x \mapsto x - x_n - \frac{h}{2} \left(f(t_{n+1},x)+f(t_n,x_n)\right)
$$

**Here we give a possible implementation of the backward Euler method**

In [33]:
def backEuler(f, t0, tf, h, x0):
    
    nt = int(round((tf-t0)/h))+1 #number of column of the output array
    t = t0 #intial time
    x = np.zeros((nt,))
    x[0] = x0 #intial value
    it = 0
    
    def g(x,tnp1,xn):
        return x-xn-h*f(tnp1,x)
    
    for it in range(nt-1):
        tn = t0+it*h
        xstart = x[it]+h*f(tn,x[it]) #initial guess for the root function
        sol = root(g,xstart,(tn+h,x[it]))
        x[it+1] = sol.x
    
    return x

**Q7. Using the function root of scipy.optimize, implement the Trapezoidal Rule method for a generic IVP associated to a function $f$**

In [40]:
def TR(f, t0, tf, h, x0):
    
    nt = int(round((tf-t0)/h))+1 #number of column of the output array
    x = np.zeros((nt,)) 
    t = [t0 + i*h for i in range(nt)] 
    
    def g(x, xn, tn, tn1): 
        return x - xn - (h/2)*(f(tn1, x) + f(tn, xn)) 
    
    for i in range(nt): 
        if (i == 0): 
            x[i] = x0 
        else: 
            xstart = x[i-1] + h*f(t[i-1],x[i-1]) #initial guess for the root function
            sol = root(g,xstart,(t[i], t[i-1], x[i-1])) 
            x[i] = sol.x
    
    return x

**Q8. Test these functions with IVP (1) and (2). For the Trapezoidal rule method, compare this implementation to the first one.**

In [41]:
t0 = 0.
tf = 4.0
h = 0.2
x0 = 1.

x_BE  = backEuler(f2, t0, tf, h, x0) 
x_TR  = TR(f2, t0, tf, h, x0) 

nt = np.size(x_BE) 
t = t0+1.*h*np.arange(nt)

N = 100 # number of points for the exact solution
texa = np.linspace(t0,tf,N)
xexa1 = x2_exa(texa)

fig1 = figure(x_range=(t0, tf), width=980, height=400, x_axis_label='t', y_axis_label='x(t)')
fig1.line(texa,xexa1,legend_label = 'Exact solution')
fig1.circle(t,x_BE, color = 'navy', legend_label = 'Back Euler method approximation', size = 8)
fig1.circle(t,x_TR, fill_color = 'white', legend_label = 'Trapezoidal Rule method approximation', size = 8)

xexa2 = x2_exa(t)
fig2 = figure(x_range=(t0, tf), width=980, height=400, x_axis_label='t', y_axis_label='GE')
fig2.circle(t,abs(xexa2-x_BE), color = 'navy', legend_label = 'Back Euler method approximation', size = 8)
fig2.circle(t,abs(xexa2-x_TR), fill_color = 'white', legend_label = 'Trapezoidal Rule method approximation', size = 8)

show(column(fig1,fig2)) 

## General implicit 2-step methods

We recall that a general implicit 2-step method is of the form, for $n\in\mathbb{N}$:
$$
x_{n+2} + \alpha_1 x_{n+1} + \alpha_0 x_n = h\left(\beta_2f(t_{n+2},x_{n+2}) +\beta_1 f(t_{n+1},x_{n+1})+\beta_0 f(t_n,x_n)\right)
$$
For this method to be consistent, it is required that the following equalities hold:
$$
1+\alpha_1+\alpha_0 = 0, \quad 2+\alpha_1 = \beta_2+\beta_1+\beta_0
$$

**Q9. Implement a general method that takes as argument the IVP function $f$, the initial and final times $t_0$ and $t_f$, the step size $h$, the starting point $x_0$ and method coefficients $a_k = [\alpha_0,\alpha_1]$ and $b_k = [\beta_0, \beta_1, \beta_2]$. The first step $x_1$ of the method should be obtained thanks to a chosen method $strt\_mthd$.**

In [ ]:
def LMMimp(f,t0, tf, h, x0, ak, bk, strt_mthd): 
    
    nt = int(round((tf-t0)/h))+1 #number of column of the output array
    x = np.zeros((nt,))
    
    #########################
    #To be completed
    #########################
    
    return x

**Q10. Test this new function with previous implementations of Backward Euler and Trapezoidal rule (Adam-Moulton methods respectively for p=-1 and p=0).**

**Q11. Study the order of convergence of Backward Euler, Trapezoidal rule, third-order Adams-Moulton (p=1) and Simpson's rule. For Adams-Moulton and Simpson's rule use the Trapezoidal rule method for the first step $x_1$.
Is the theoretical order of convergence reached for the Simpson's rule?**

**Q12. propose an explanation as to why the Simpson's rule is not of the expected order. Subsequently, implement a modification that makes the Simpson's rule convergent of order 4.**